In [1]:
%matplotlib notebook
import torch
import pandas as pd
from torch.utils.data import TensorDataset, DataLoader
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import scipy
from scipy import signal
import numpy as np

In [2]:
annotation = pd.read_pickle('/media/hticpose/drive1/Prithvi/PPG/PPG_FieldStudy/annotation.pkl')
input_data = torch.load('/media/hticpose/drive1/Prithvi/PPG/PPG_FieldStudy/input_signal.pt')
output_data = torch.load('/media/hticpose/drive1/Prithvi/PPG/PPG_FieldStudy/output_signal.pt')
raw_signal = torch.load('/media/hticpose/drive1/Prithvi/PPG/PPG_FieldStudy/raw_signals.pt')
training_ids = annotation['patient_id'] < 13
reference_rr = (annotation['Reference_RR'].values).reshape(-1,1)
torch_ref_rr = torch.from_numpy(reference_rr)
train_ids = annotation.loc[annotation['patient_id']<13]
test_ids = annotation.loc[annotation['patient_id']>=13]

x_train_data = input_data[torch.from_numpy(training_ids.values)]
x_test_data = input_data[torch.from_numpy(~(training_ids.values))]
y_train_data = output_data[torch.from_numpy(training_ids.values)]
y_test_data = output_data[torch.from_numpy(~(training_ids.values))]

x_train_ref_rr = torch_ref_rr[torch.from_numpy(training_ids.values)]
x_test_ref_rr = torch_ref_rr[torch.from_numpy(~(training_ids.values))]
raw_train_signals = raw_signal[torch.from_numpy(training_ids.values)]
raw_test_signals = raw_signal[torch.from_numpy(~(training_ids.values))]

In [3]:
torch_train_resp = TensorDataset(x_train_data , y_train_data)
trainloader_resp = DataLoader(torch_train_resp, batch_size = 128 , shuffle=True)

torch_test_resp = TensorDataset(x_test_data , y_test_data)
testloader_resp = DataLoader(torch_test_resp, batch_size = 128 , shuffle=False)
#-----------------------------------------------------------------------------------------
torch_train_rr = TensorDataset(x_train_data , x_train_ref_rr)
trainloader_rr = DataLoader(torch_train_rr, batch_size = 128 , shuffle=True)

torch_test_rr = TensorDataset(x_test_data , x_test_ref_rr)
testloader_rr = DataLoader(torch_test_rr, batch_size = 128 , shuffle=False)
#-----------------------------------------------------------------------------------------
torch_train_resp_rr = TensorDataset(x_train_data ,y_train_data, x_train_ref_rr)
trainloader_resp_rr = DataLoader(torch_train_resp_rr, batch_size = 128 , shuffle=True)

torch_test_resp_rr = TensorDataset(x_test_data ,y_test_data, x_test_ref_rr)
testloader_resp_rr = DataLoader(torch_test_resp_rr, batch_size = 128 , shuffle=False)
#-----------------------------------------------------------------------------------------
torch_train_ecg_resp = TensorDataset(raw_train_signals , y_train_data)
trainloader_ecg_resp = DataLoader(torch_train_ecg_resp, batch_size = 128 , shuffle=True)

torch_test_ecg_resp = TensorDataset(raw_test_signals , y_test_data)
testloader_ecg_resp = DataLoader(torch_test_ecg_resp, batch_size = 128 , shuffle=False)
#-----------------------------------------------------------------------------------------
torch_train_ecg_rr = TensorDataset(raw_train_signals , x_train_ref_rr)
trainloader_ecg_rr = DataLoader(torch_train_ecg_rr, batch_size = 128 , shuffle=True)

torch_test_ecg_rr = TensorDataset(raw_test_signals , x_test_ref_rr)
testloader_ecg_rr = DataLoader(torch_test_ecg_rr, batch_size = 128 , shuffle=False)
#----------------------------------------------------------------------------------------
torch_train_ecg_rr_resp = TensorDataset(raw_train_signals , y_train_data,x_train_ref_rr)
trainloader_ecg_rr_resp = DataLoader(torch_train_ecg_rr_resp, batch_size = 128 , shuffle=True)

torch_test_ecg_rr_resp = TensorDataset(raw_test_signals , y_test_data,x_test_ref_rr)
testloader_ecg_rr_resp = DataLoader(torch_test_ecg_rr_resp, batch_size = 128 , shuffle=False)


In [4]:
best_model = torch.load('/media/hticpose/drive1/Prithvi/PPG/PPG_FieldStudy/SIGNAL_0.005/2021_03_27_11_47/model.pt')
best_model_parameters = torch.load('/media/hticpose/drive1/Prithvi/PPG/PPG_FieldStudy/SIGNAL_0.005/2021_03_27_11_47/parameters.pt')
best_model.eval()
test_loss = []
final_output = torch.tensor([]).cuda()
criterion = torch.nn.SmoothL1Loss()
for i, (x_resp,y_resp) in enumerate(testloader_resp):
    x_resp = x_resp.float().cuda()
    y_resp = y_resp.float().view(-1,1,y_resp.shape[-1]).cuda() 
    model_output = best_model(x_resp)
    model_output = model_output.view(-1,1,model_output.shape[-1])
    #import pdb;pdb.set_trace()
    loss = criterion(y_resp, model_output)
    final_output = torch.cat((final_output , model_output) , dim= 0 )
    test_loss.append(float(loss.cpu().data))
final_resp_sig = final_output.detach().cpu().numpy()
final_resp_sig = final_resp_sig.reshape(final_resp_sig.shape[0] , final_resp_sig.shape[2])
ref_resp_sig = y_test_data.detach().cpu().numpy()
pytorch_total_params = sum(p.numel() for p in best_model.parameters())
print(pytorch_total_params)

20943613


In [5]:
def extremas_extraction(signal):
    avg_breath_duration = np.array([])
    extrema_relevent = []
    for item in signal:
        amplitude = np.array([])
        pos_peaks , _ = scipy.signal.find_peaks(item , height = [-300,300])
        neg_peaks , _ = scipy.signal.find_peaks(-1*item , height = [-300 , 300])
        extremas = np.concatenate((pos_peaks , neg_peaks))
        extremas = np.sort(extremas)
        for i in range(len(extremas)):
            amplitude = np.append(amplitude , item[int(extremas[i])])
        amplitude_diff = np.abs(np.diff(amplitude))
        q3 = np.percentile(amplitude_diff , 75)
        threshold = 0.3*q3
        eliminate_pairs_of_extrema = 1
        while(eliminate_pairs_of_extrema):
            amps = np.array([])
            if len(extremas)<3:
                eliminate_pairs_of_extrema = 0
                continue
            for i in range(len(extremas)):
                amps = np.append(amps , item[int(extremas[i])])
            amp_diff = np.abs(np.diff(amps)) 
            min_amp_diff , index = min(amp_diff) , np.argmin(amp_diff)
            #print(min_amp_diff)
            if min_amp_diff > threshold:
                eliminate_pairs_of_extrema = 0
                #extrema_relevent = extremas
            else:
                extremas = np.concatenate((extremas[0:index] , extremas[index+2 :]))
                #amplitude_diff = np.delete(amplitude_diff , index)
        if item[int(extremas[0])] < item[int(extremas[1])]:
            extremas = extremas[1:]
        if item[int(extremas[-1])] < item[int(extremas[-2])]:
            extremas = extremas[:-1]
        no_of_breaths = (len(extremas)-1)/2
        breath_duration = extremas[-1] - extremas[0]
        avg_breath_duration = np.append(avg_breath_duration , breath_duration/no_of_breaths)
        extrema_relevent.append(extremas)
    return avg_breath_duration , extrema_relevent



In [24]:
fbpB,fbpA = scipy.signal.butter(6,np.array([0.1,0.7])/2,btype='bandpass')
filt_sig = []
filt_ref_sig = []
for item in final_resp_sig:
    filt_sig.append(scipy.signal.filtfilt(fbpB,fbpA,item))
for item in ref_resp_sig:
    filt_ref_sig.append(scipy.signal.filtfilt(fbpB,fbpA,item))

In [25]:
duration_resp,extremas_resp = extremas_extraction(filt_sig)
duration_ref_resp,extremas_ref_resp = extremas_extraction(filt_ref_sig)

In [26]:
rr_resp = 240/duration_resp
rr_ref = 240/duration_ref_resp

In [27]:
error = np.abs(rr_resp - rr_ref)

In [28]:
mae = np.mean(error)
rmse = np.sqrt(np.mean(error**2))

In [29]:
mae

2.668680240439738

In [30]:
best_model = torch.load('/media/hticpose/drive1/Prithvi/PPG/PPG_FieldStudy/ENCODER_ONLY_0.005/2021_03_26_14_46/model.pt')
best_model_parameters = torch.load('/media/hticpose/drive1/Prithvi/PPG/PPG_FieldStudy/ENCODER_ONLY_0.005/2021_03_26_14_46/parameters.pt')
best_model.eval()
test_loss = []
final_output_rr = torch.tensor([]).cuda()
criterion = torch.nn.SmoothL1Loss()
for i, (x_resp,y_rr) in enumerate(testloader_rr):
    x_resp = x_resp.float().cuda()
    y_rr = y_rr.float().cuda()
    out_rr = best_model(x_resp)
    #out_rr = out_rr.view(-1,1,model_output.shape[-1])
    loss = 0.01*criterion(y_rr, out_rr)
    final_output_rr = torch.cat((final_output_rr , out_rr) , dim= 0 )
    test_loss.append(float(loss.cpu().data))

output_rr = final_output_rr.detach().cpu().numpy()
pytorch_total_params = sum(p.numel() for p in best_model.parameters())
ref_rr = x_test_ref_rr.detach().cpu().numpy()
print(pytorch_total_params)

16647332


In [31]:
error = np.abs(output_rr - rr_ref)

In [32]:
mae = np.mean(error)
rmse = np.sqrt(np.mean(error**2))

In [33]:
rmse

4.418603258612834

In [34]:
best_model = torch.load('/media/hticpose/drive1/Prithvi/PPG/PPG_FieldStudy/ENCODER_SIGNAL_MULTI_0.005/2021_03_28_17_10/model.pt')
best_model_parameters = torch.load('/media/hticpose/drive1/Prithvi/PPG/PPG_FieldStudy/ENCODER_SIGNAL_MULTI_0.005/2021_03_28_17_10/parameters.pt')
best_model.eval()
test_loss = []
final_output_rr = torch.tensor([]).cuda()
final_output = torch.tensor([]).cuda()
criterion = torch.nn.SmoothL1Loss()
criterion_rr = torch.nn.SmoothL1Loss()
for i, (x_resp,y_resp,y_rr) in enumerate(testloader_resp_rr):
    x_resp = x_resp.float().cuda()
    y_resp = y_resp.float().view(-1,1,y_resp.shape[-1]).cuda() 
    y_rr = y_rr.float().cuda()
    output_resp , out_rr = best_model(x_resp)
    #out_rr = out_rr.view(-1,1,model_output.shape[-1])
    loss_resp = criterion(y_resp, output_resp)
    loss_rr = 0.01*criterion_rr(y_rr,out_rr)
    total_loss = loss_resp+loss_rr
    final_output_rr = torch.cat((final_output_rr , out_rr) , dim= 0 )
    final_output = torch.cat((final_output , output_resp) , dim= 0 )
    test_loss.append(float(total_loss.cpu().data))

output_rr = final_output_rr.detach().cpu().numpy()
final_resp_sig = final_output.detach().cpu().numpy()
final_resp_sig = final_resp_sig.reshape(final_resp_sig.shape[0] , final_resp_sig.shape[2])
ref_resp_sig = y_test_data.detach().cpu().numpy()
pytorch_total_params = sum(p.numel() for p in best_model.parameters())
print(pytorch_total_params)

21688913


In [35]:
fbpB,fbpA = scipy.signal.butter(5,np.array([0.1,0.6])/2,btype='bandpass')
filt_sig = []
filt_ref_sig = []
for item in final_resp_sig:
    filt_sig.append(scipy.signal.filtfilt(fbpB,fbpA,item))
for item in ref_resp_sig:
    filt_ref_sig.append(scipy.signal.filtfilt(fbpB,fbpA,item))

In [36]:
duration_resp,extremas_resp = extremas_extraction(filt_sig)
duration_ref_resp,extremas_ref_resp = extremas_extraction(filt_ref_sig)

In [37]:
rr_resp = 240/duration_resp
rr_ref = 240/duration_ref_resp

In [38]:
error = np.abs(output_rr - rr_ref)

In [39]:
mae = np.mean(error)
rmse = np.sqrt(np.mean(error**2))

In [40]:
mae

2.927675917397163

In [41]:
rmse

3.6580598238000075